In [40]:
import os 
import numpy as np
import scipy
from scipy import stats
import matplotlib.pyplot as plt
import pandas as pd 
import seaborn as sns
import statsmodels.api as sm
from sklearn.model_selection import train_test_split

In [2]:
datadir = "~/Sail/Dianbodataset/Archive 2"

In [3]:
df1 = pd.read_csv(f"{datadir}/drug_table_mortality_with_no.csv")
df2 = pd.read_csv(f"{datadir}/mortality_table.csv")
df3 = df1.set_index("SUBJECT_ID").join(df2).drop("SUBJECT_ID",axis = 1).fillna(0)

In [4]:
class LogRegCV:

    def __init__(self, penalty_range = [0.1,1,10,100,1000], penalty='l1', lbd = 10, lr=0.01, num_iter=1000, fit_intercept=True, DP = None):

        self.penalty = penalty
        self.penalty_range = penalty_range
        self.lbd = lbd 
        self.lr = lr
        self.num_iter = num_iter
        self.fit_intercept = fit_intercept
        self.DP = DP 
    
    def __add_intercept(self, X): # add intercept 
        intercept = np.ones((X.shape[0], 1))
        return np.concatenate((intercept, X), axis=1)
    
    def __sigmoid(self, z):
        return 1 / (1 + np.exp(-z)) # Maybe add 0.001 

    def RSS(self,X_Test,Y_Test):
        
        if self.fit_intercept: #we make a condition on the input of the class 
            X_Test = self.__add_intercept(X_Test)
    
        predict_prob = self.__sigmoid(np.array(np.dot(self.theta,X_Test.T),dtype = float))
        return(np.mean((predict_prob - np.reshape(Y_Test,(1,len(Y_Test))))**2))

    def RSS_DP1(self,X_Test,Y_Test,eps):
        
        if self.fit_intercept: #we make a condition on the input of the class 
            X_Test = self.__add_intercept(X_Test)

        n = X_Test.shape[0]
        norm_etha = np.random.gamma(len(self.theta), 2/(n*eps*self.lbd),1)
        noise = np.random.laplace(0, 2/(n*eps*self.lbd*abs(norm_etha)),self.theta.shape)
        self.theta = self.theta + noise # adjusting the weight

        predict_prob = self.__sigmoid(np.array(np.dot(self.theta,X_Test.T),dtype = float))
        return(np.mean((predict_prob - np.reshape(Y_Test,(1,len(Y_Test))))**2))

    def fitCV_DP1(self, X, y,X_Test,Y_Test,eps):
        
        RSS = []
        RSS_DP = []

        if self.fit_intercept: #we make a condition on the input of the class IS it possible that we repeat this function two times? 
            X = self.__add_intercept(X)
        # weights initialization 
        for reg in self.penalty_range:
            self.lbd = reg

            self.theta = np.random.normal(0,1,size = X.shape[1])

            if self.penalty == None or self.penalty == 0: 
                
                for i in range(self.num_iter):
                    z = np.array(np.dot(X, self.theta),dtype=np.float32)
                    h = self.__sigmoid(z)
                    h = np.reshape(h,(len(h),1))
                    gradient = np.mean((y - h)*X , axis = 0) 
                    self.theta = self.theta - self.lr * gradient
                RSS.append(self.RSS(X_Test,Y_Test))
                RSS_DP.append(self.RSS_DP1(X_Test,Y_Test,eps))

            if self.penalty == 'l1': 

                for i in range(self.num_iter):
                    grad_l1_theta = np.sign(self.theta)
                    z = np.array(np.dot(X, self.theta),dtype=np.float32)
                    h = self.__sigmoid(z)
                    h = np.reshape(h,(len(h),1))
                    gradient = np.mean((y - h)*X, axis = 0) + grad_l1_theta*self.lbd
                    self.theta = self.theta - self.lr * gradient

                RSS.append(self.RSS(X_Test,Y_Test))
                RSS_DP.append(self.RSS_DP1(X_Test,Y_Test,eps))

            if self.penalty == 'l2': 

                for i in range(self.num_iter):
                    z = np.array(np.dot(X, self.theta),dtype=np.float32)
                    h = self.__sigmoid(z)
                    h = np.reshape(h,(len(h),1))
                    gradient = np.mean((y - h)*X, axis = 0) + 2*(self.lbd*np.asarray(self.theta))
                    self.theta = self.theta - self.lr * gradient

                RSS.append(self.RSS(X_Test,Y_Test))
                RSS_DP.append(self.RSS_DP1(X_Test,Y_Test,eps))
                
        return RSS, RSS_DP, [self.penalty_range[RSS.index(min(RSS))], min(RSS)], [self.penalty_range[RSS_DP.index(min(RSS_DP))], min(RSS_DP)]
        print ("optimal Regularizer is" + str(self.penalty_range[RSS.index(min(RSS))]))
        print("RSS is " + str(min(RSS)))
        print ("optimal DP_Regularizer is" + str(self.penalty_range[RSS_DP.index(min(RSS_DP))]))
        print("RSS_DP is " + str(min(RSS_DP)))

# NEED TO THINK OF BEING ABLE VARIATE EPSILON !!!!!!!!!!!! 

    def fitCV_DP2(self, X, y,X_Test,Y_Test,eps):

        RSS_DP = []
        if self.fit_intercept: #we make a condition on the input of the class 
            X = self.__add_intercept(X)
        # weights initialization
        
        for reg in self.penalty_range:
            self.lbd = reg

            self.theta = np.random.normal(0,1,size = X.shape[1])
            n = X.shape[0]
            norm_b = np.random.gamma(11, 2/(eps),1)
            self.b = np.random.laplace(0, 2/(eps*abs(norm_b)),self.theta.shape)
        
            if self.penalty == None or self.penalty == 0: 
                
                for i in range(self.num_iter):
                    z = np.array(np.dot(X, self.theta),dtype=np.float32)
                    h = self.__sigmoid(z)
                    h = np.reshape(h,(len(h),1))
                    gradient = np.mean((y - h)*X, axis = 0) + self.b /y.size
                    self.theta = self.theta - self.lr * gradient
                RSS_DP.append(self.RSS(X_Test,Y_Test))
            
            if self.penalty == 'l1': 

                for i in range(self.num_iter):
                    grad_l1_theta = np.sign(self.theta)
                    z = np.array(np.dot(X, self.theta),dtype=np.float32)
                    h = self.__sigmoid(z)
                    h = np.reshape(h,(len(h),1))
                    gradient = np.mean((y - h)*X, axis = 0) + grad_l1_theta*self.lbd + self.b /y.size
                    self.theta = self.theta - self.lr * gradient
                RSS_DP.append(self.RSS(X_Test,Y_Test))

            if self.penalty == 'l2': 

                for i in range(self.num_iter):
                    z = np.array(np.dot(X, self.theta),dtype=np.float32)
                    h = self.__sigmoid(z)
                    h = np.reshape(h,(len(h),1))
                    gradient = np.mean((y - h)*X, axis = 0) + 2*(self.lbd*np.asarray(self.theta)) + self.b /y.size
                    self.theta = self.theta - self.lr * gradient
                RSS_DP.append(self.RSS(X_Test,Y_Test))

        return RSS_DP,[self.penalty_range[RSS_DP.index(min(RSS_DP))], min(RSS_DP)]
        print ("optimal DP_Regularizer is" + str(self.penalty_range[RSS_DP.index(min(RSS_DP))]))
        print("RSS_DP is " + str(min(RSS_DP)))
                        
    def predict_prob(self, X):
        return self.__sigmoid(np.array(np.dot(X,self.theta[1:]) + self.theta[0] ,dtype = float)) 
    
    def predict(self, X, threshold):
        Y = []
        for i in X:
        
            if self.predict_prob(i) >= threshold: 
                Y.append(1)
            else: 
                Y.append(0)
        return np.asarray(Y)

    def F1_score(self,X_Test,Y_Test):
        
        Y_Test = np.reshape(Y_Test,(1,len(Y_Test)))[0]
        Predict = self.predict(X_Test,0.5)

        P = np.sum(Predict[Y_Test==1] == Y_Test[Y_Test==1]) / (np.sum(Predict[Y_Test==1] == Y_Test[Y_Test==1]) +\
                                                     np.sum(Predict[Y_Test == 0])) # False positive
        R = np.sum(Predict[Y_Test==1] == Y_Test[Y_Test==1]) / (np.sum(Predict[Y_Test==1] == Y_Test[Y_Test==1]) +\
                                                     np.sum(Y_Test[Predict==0])) # False negative
    
        return((2*P*R)/(P+R))
    
    def Score(self,X_Test,Y_Test): 
        Predict = self.predict(X_Test,0.5)
        Y_Test = np.reshape(Y_Test,(1,len(Y_Test)))[0]
        return(np.sum(Predict == Y_Test) / len(Y_Test))
    
    def classifier():
        return (self.theta)

In [5]:
X_Train, X_Test, Y_Train, Y_Test = train_test_split(np.asarray(df1.drop("SUBJECT_ID",axis = 1)),\
                                np.asarray(df2.drop("SUBJECT_ID",axis = 1)),test_size=0.3, random_state = 0)

X_Train, X_Val, Y_Train, Y_Val = train_test_split(X_Train,Y_Train, random_state = 0)

In [47]:
model = LogRegCV(penalty_range = [0.1,1,10,100,1000],penalty= 'l1' ,num_iter=2000, DP ='DP2')

In [48]:
RSS, RSS_DP, a,b = model.fitCV_DP1(X_Train,Y_Train,X_Val, Y_Val,0.5)

In [50]:
RSS,RSS_DP

([0.6834472895796396,
  0.25034808642922374,
  0.2500224514927245,
  0.2573875921899066,
  0.6827770395547981],
 [0.6834472889289637,
  0.25027452751897533,
  0.2500892160048673,
  0.2574379904742989,
  0.6827771045998386])

In [70]:
model = LogRegCV(penalty_range = [0.1,1,10,100,1000],penalty= 'l1' ,num_iter=500, DP ='DP1')

In [71]:
%time model.fitCV_DP1(X_Train,Y_Train,X_Val, Y_Val,0.05)

CPU times: user 9min 37s, sys: 6min 11s, total: 15min 48s
Wall time: 14min 29s


([0.6828358221850207,
  0.25082685252340475,
  0.2633628021866038,
  0.3794172587220339,
  0.6342542273542804],
 [0.682837014635114,
  0.2508150974875983,
  0.2634476103690502,
  0.3793980513723284,
  0.6342595098491597],
 [1, 0.25082685252340475],
 [1, 0.2508150974875983])

In [54]:
RSS1, RSS_DP_1

([0.6744000368960498,
  0.25258625902593584,
  0.27697526251150645,
  0.3282463834851715,
  0.6470420585670378],
 [0.6744042640366004,
  0.25261205137624687,
  0.2769713191176171,
  0.32823839816584033,
  0.6470065367790393])

In [63]:
model = LogRegCV(penalty_range = [0.1,1,10,100,1000],penalty= 'l1' ,num_iter=500, DP ='DP2')

In [64]:
RSS_DP2,a_DP2 = model.fitCV_DP2(X_Train,Y_Train,X_Val, Y_Val,0.5)

In [66]:
RSS_DP2, a_DP2

([0.6795205148343296,
  0.25179813693154895,
  0.28316560872314106,
  0.46179423535921854,
  0.5948717712082529],
 [1, 0.25179813693154895])

# Cross Validation; Optimal Lambda : Range [0.1,1,10,100,1000]

In [55]:
Eps1 = np.linspace(0.001,0.1,20)

RSS_lbd_01,RSS_lbd_1,RSS_lbd_10,RSS_lbd_100,RSS_lbd_1000 = [],[],[],[],[]
RSS_DP1_lbd_01,RSS_DP1_lbd_1,RSS_DP1_lbd_10,RSS_DP1_lbd_100,RSS_DP1_lbd_1000 = [],[],[],[],[]
RSS_DP2_lbd_01,RSS_DP2_lbd_1,RSS_DP2_lbd_10,RSS_DP2_lbd_100,RSS_DP2_lbd_1000 = [],[],[],[],[]
Lbd_RSS_Opt, Lbd_RSS_Opt,Lbd_RSS_DP1_Opt, Lbd_RSS_DP1_Opt,Lbd_RSS_DP2_Opt, Lbd_RSS_DP2_Opt = [],[],[],[],[],[]



for i in Eps1:
    
    model = LogRegCV(penalty_range = [0.1,1,10,100,1000],penalty= 'l1' ,num_iter=150, DP ='DP1')
    RSS, RSS_DP1, a,b = model.fitCV_DP1(X_Train,Y_Train,X_Val, Y_Val,i)
    
    model = LogRegCV(penalty_range = [0.1,1,10,100,1000],penalty= 'l1' ,num_iter=150, DP ='DP2')
    RSS_DP2,a_DP2 = model.fitCV_DP2(X_Train,Y_Train,X_Val, Y_Val,i)
    
    RSS_lbd_01.append(RSS[0])
    RSS_lbd_1.append(RSS[1])
    RSS_lbd_10.append(RSS[2])
    RSS_lbd_100.append(RSS[3])    
    RSS_lbd_1000.append(RSS[4])
    
    RSS_DP1_lbd_01.append(RSS_DP1[0])
    RSS_DP1_lbd_1.append(RSS_DP1[1])
    RSS_DP1_lbd_10.append(RSS_DP1[2])
    RSS_DP1_lbd_100.append(RSS_DP1[3])
    RSS_DP1_lbd_1000.append(RSS_DP1[4])
    
    RSS_DP2_lbd_01.append(RSS_DP2[0])
    RSS_DP2_lbd_1.append(RSS_DP2[1])
    RSS_DP2_lbd_10.append(RSS_DP2[2])
    RSS_DP2_lbd_100.append(RSS_DP2[3])
    RSS_DP2_lbd_1000.append(RSS_DP2[4])
    
    Lbd_RSS_Opt.append(a)
    Lbd_RSS_DP1_Opt.append(b)
    Lbd_RSS_DP2_Opt.append(a_DP2)

In [56]:
Opt_Lbd,Opt_RSS,Opt_Lbd_DP1, Opt_RSS_DP1,Opt_Lbd_DP2, Opt_RSS_DP2 = [],[],[],[],[],[]

for j in range (0,len(Lbd_RSS_Opt)): 
    Opt_Lbd.append(Lbd_RSS_Opt[j][0])
    Opt_RSS.append(Lbd_RSS_Opt[j][1])
    
    Opt_Lbd_DP1.append(Lbd_RSS_DP1_Opt[j][0])
    Opt_RSS_DP1.append(Lbd_RSS_DP1_Opt[j][1])
    
    Opt_Lbd_DP2.append(Lbd_RSS_DP2_Opt[j][0])
    Opt_RSS_DP2.append(Lbd_RSS_DP2_Opt[j][1])

d = {"epsilon": Eps1 ,  "Optimal lbd " : Opt_Lbd , "Optimal RSS": Opt_RSS , "Optimal lbd DP1": Opt_Lbd_DP1,\
     
    "Optimal RSS DP1": Opt_RSS_DP1, "Optimal lbd DP2": Opt_Lbd_DP2, "Optimal RSS DP2": Opt_RSS_DP2}
df = pd.DataFrame(data = d)

In [57]:
df.to_csv("1stResultLogRegCV0_001.csv",sep='\t')

In [58]:
d1 = {"epsilon": Eps1,\
      "RSS_lbd_01" : RSS_lbd_01 , "RSS_lbd_1": RSS_lbd_1 , "RSS_lbd_10": RSS_lbd_10,\
      "RSS_lbd_100": RSS_lbd_100, "RSS_lbd_1000": RSS_lbd_1000,\
      "RSS_DP1_lbd_01" : RSS_DP1_lbd_01 , "RSS_DP1_lbd_1": RSS_DP1_lbd_1 , "RSS_DP1_lbd_10": RSS_DP1_lbd_10,\
      "RSS_DP1_lbd_100": RSS_DP1_lbd_100, "RSS_DP1_lbd_1000": RSS_DP1_lbd_1000,\
     "RSS_DP2_lbd_01" : RSS_DP2_lbd_01 , "RSS_DP2_lbd_1": RSS_DP2_lbd_1 , "RSS_DP2_lbd_10": RSS_DP2_lbd_10,\
      "RSS_DP2_lbd_100": RSS_DP2_lbd_100, "RSS_DP2_lbd_1000": RSS_DP2_lbd_1000}
df1 = pd.DataFrame(data = d1)

In [59]:
df1.to_csv("CVRSSresults1st0_001.csv")

In [61]:
df1

,epsilon,RSS_lbd_01,RSS_lbd_1,RSS_lbd_10,RSS_lbd_100,RSS_lbd_1000,RSS_DP1_lbd_01,RSS_DP1_lbd_1,RSS_DP1_lbd_10,RSS_DP1_lbd_100,RSS_DP1_lbd_1000,RSS_DP2_lbd_01,RSS_DP2_lbd_1,RSS_DP2_lbd_10,RSS_DP2_lbd_100,RSS_DP2_lbd_1000
0,0.001000,0.310591,0.350812,0.266724,0.522925,0.366878,0.310589,0.350914,0.266704,0.522928,0.366902,0.336118,0.275156,0.278909,0.278739,0.360452
1,0.006211,0.607536,0.285157,0.254893,0.341582,0.585787,0.607598,0.285140,0.254896,0.341627,0.585825,0.572893,0.496394,0.237474,0.365072,0.513610
2,0.011421,0.385301,0.312123,0.268665,0.242514,0.398054,0.385330,0.312108,0.268442,0.242510,0.398065,0.637855,0.274197,0.250341,0.349745,0.535373
3,0.016632,0.625933,0.404093,0.236906,0.267239,0.488259,0.625940,0.404198,0.236884,0.267233,0.488271,0.517146,0.256652,0.268869,0.337983,0.418664
4,0.021842,0.571730,0.455073,0.272806,0.353072,0.501994,0.571676,0.454989,0.272536,0.353044,0.501999,0.656193,0.295458,0.259657,0.329251,0.391647
5,0.027053,0.443742,0.261786,0.237706,0.308988,0.371626,0.443760,0.261725,0.237690,0.308990,0.371656,0.522438,0.283790,0.285874,0.450298,0.565001
6,0.032263,0.323158,0.459750,0.281326,0.376320,0.622277,0.323152,0.459734,0.281291,0.376249,0.622295,0.387215,0.288382,0.248522,0.302406,0.402958
7,0.037474,0.308413,0.284973,0.254112,0.322173,0.566852,0.308404,0.284991,0.254037,0.322185,0.566823,0.452407,0.354589,0.264909,0.449760,0.460117
8,0.042684,0.397383,0.432462,0.251175,0.368294,0.570480,0.397349,0.432577,0.251037,0.368280,0.570483,0.582542,0.332039,0.239363,0.343794,0.377583
9,0.047895,0.418665,0.310884,0.256730,0.332558,0.404483,0.418685,0.310867,0.256624,0.332499,0.404497,0.673594,0.374988,0.273834,0.417685,0.481555
